In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from random import randint
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
import datetime
import csv
import numpy as np
import random
import torch
import torch.nn as nn
import warnings
import pandas as pd
import torch.nn.functional as F
from copy import deepcopy

torch.manual_seed(2021)

#ratings = pd.read_csv("/content/drive/MyDrive/Copy of Epinions/Data-final/ratings_data.csv")
#ratings = pd.read_csv("/content/drive/MyDrive/Copy of Epinions/Ciao/ratings_data.csv")
#ratings = ratings[['userId', 'productId', 'rating']].values

#print(len(np.unique(ratings[:, 0])), len(np.unique(ratings[:, 1])))

train = pd.read_csv("/content/drive/MyDrive/Implicit/Ciao/trainImplicit.csv")
train = train[["userId","productId","rating"]]
train
#train = pd.read_csv("/content/drive/MyDrive/Copy of Epinions/Ciao/ratings_data_train.csv")
#train = train[["userId","productId","rating"]].values

print(train.shape)

(181913, 3)


In [ ]:
from torch.utils.data import DataLoader,Dataset

class UserItemRatingDataset(Dataset):
    """Wrapper, convert <user, item, rating> Tensor into Pytorch Dataset"""
    def __init__(self, user_tensor, item_tensor, target_tensor):
        """
        args:
            target_tensor: torch.Tensor, the corresponding rating for <user, item> pair
        """
        self.user_tensor = user_tensor
        self.item_tensor = item_tensor
        self.target_tensor = target_tensor

    def __getitem__(self, index):
      get_output = [self.user_tensor[index], self.item_tensor[index], self.target_tensor[index]]

      return get_output

    def __len__(self):
        return self.user_tensor.size(0)

In [ ]:
def instance_a_train_loader(data, batch_size):
        users = []
        items = []
        ratings = []

        for row in data:
            users.append(int(row[0]))
            items.append(int(row[1]))
            ratings.append(float(row[2]))
        dataset = UserItemRatingDataset(user_tensor=torch.LongTensor(users),
                                        item_tensor=torch.LongTensor(items),
                                        target_tensor=torch.FloatTensor(ratings),
                                        )
        
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
train_loader = instance_a_train_loader(train.values,1000)

In [ ]:
import torch
import torch.nn as nn

class Generalized_Matrix_Factorization(nn.Module):
    def __init__(self, args, num_users, num_items):
        super(Generalized_Matrix_Factorization, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num = args["factor_num"]

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num)

        self.affine_output = nn.Linear(in_features=self.factor_num, out_features=1)
        self.logistic = nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        element_product = torch.mul(user_embedding, item_embedding)
        logits = self.affine_output(element_product)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        pass

class Multi_Layer_Perceptron(nn.Module):
    def __init__(self, args, num_users, num_items):
        super(Multi_Layer_Perceptron, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num = args["factor_num"]
        self.layers = args["layers"]

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num)

        self.fc_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(self.layers[:-1], self.layers[1:])):
            self.fc_layers.append(nn.Linear(in_size, out_size))

        self.affine_output = nn.Linear(in_features=self.layers[-1], out_features=1)
        self.logistic = nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        vector = torch.cat([user_embedding, item_embedding], dim=-1)  # the concat latent vector
        for idx, _ in enumerate(range(len(self.fc_layers))):
            vector = self.fc_layers[idx](vector)
            vector = nn.ReLU()(vector)
            # vector = nn.BatchNorm1d()(vector)
            # vector = nn.Dropout(p=0.5)(vector)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        pass



class NeuMF(nn.Module):
    def __init__(self, args):
        super(NeuMF, self).__init__()
        self.num_users = args["num_users"]
        self.num_items = args["num_items"]
        self.factor_num_mf = args["factor_num"]
        self.factor_num_mlp =  int(args["layers"][0]/2)
        self.layers = args["layers"]
        self.dropout = args["dropout"]

        self.embedding_user_mlp = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num_mlp)
        self.embedding_item_mlp = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num_mlp)

        self.embedding_user_mf = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num_mf)
        self.embedding_item_mf = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num_mf)

        self.fc_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(args["layers"][:-1], args["layers"][1:])):
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))
            self.fc_layers.append(nn.Sigmoid())

        self.affine_output = nn.Linear(in_features=args["layers"][-1] + self.factor_num_mf, out_features=1)
        self.logistic = nn.Sigmoid()
        self.init_weight()

    def init_weight(self):
        nn.init.normal_(self.embedding_user_mlp.weight, std=1)
        nn.init.normal_(self.embedding_item_mlp.weight, std=1)
        nn.init.normal_(self.embedding_user_mf.weight, std=1)
        nn.init.normal_(self.embedding_item_mf.weight, std=1)
        
        for m in self.fc_layers:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                
        nn.init.xavier_uniform_(self.affine_output.weight)

        for m in self.modules():
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()

    def forward(self, user_indices, item_indices):
        #user_indices = user_indices.cuda()
        #item_indices = item_indices.cuda()
        user_embedding_mlp = self.embedding_user_mlp(user_indices)
        item_embedding_mlp = self.embedding_item_mlp(item_indices)

        user_embedding_mf = self.embedding_user_mf(user_indices)
        item_embedding_mf = self.embedding_item_mf(item_indices)

        mlp_vector = torch.cat([user_embedding_mlp, item_embedding_mlp], dim=-1)  # the concat latent vector
        mf_vector = torch.mul(user_embedding_mf, item_embedding_mf)

        for idx, _ in enumerate(range(len(self.fc_layers))):
            mlp_vector = self.fc_layers[idx](mlp_vector)
            mlp_vector = torch.nn.Sigmoid()(mlp_vector)

        vector = torch.cat([mlp_vector, mf_vector], dim=-1)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating.squeeze()

In [ ]:
import dill
config = {"seed":42,"lr":1e-3,"dropout":0.2,"batch_size":256,"epochs":30,"factor_num":8,"layers":[64,32,16,8],"num_users":7373,"num_items":91091,"latent_dim_mf":8,"latent_dim_mlp":8}
model = NeuMF(config)
#model = model.cuda()

In [ ]:
min_loss = 1e+5
breakout = 1
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
loss_fn = torch.nn.BCELoss()
for epoch in range(200):
  total_pred_loss = 0
  total_pred_loss2 = 0
  for batch_num,batch in enumerate(train_loader):
    user = batch[0]
    item = batch[1]
    rating = batch[2]
    rating = rating.cuda()

    y_pred = model.forward(user,item)
    loss = loss_fn(y_pred.view(-1),rating)  
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_pred_loss += loss.item()
  if(round(total_pred_loss/(batch_num+1),2)<round(min_loss/(batch_num+1),2)):
    min_loss = total_pred_loss
    breakout = 0
  else:
    breakout+=1
  if(breakout==10):
    break
  if(epoch%10==0):
      file = "/content/drive/MyDrive/Implicit/Ciao/Models/NCF_"+str(epoch)+"_"+str(1e-3)
      torch.save(model.state_dict(),f=file)
      print("Model Saved")
  print("Train prediction loss is ", round(total_pred_loss/(batch_num+1),2)," for epoch ",epoch)

Model Saved
Train prediction loss is  0.59  for epoch  0
Train prediction loss is  0.27  for epoch  1
Train prediction loss is  0.15  for epoch  2
Train prediction loss is  0.09  for epoch  3
Train prediction loss is  0.06  for epoch  4
Train prediction loss is  0.05  for epoch  5
Train prediction loss is  0.04  for epoch  6
Train prediction loss is  0.03  for epoch  7
Train prediction loss is  0.02  for epoch  8
Train prediction loss is  0.02  for epoch  9
Model Saved
Train prediction loss is  0.02  for epoch  10
Train prediction loss is  0.01  for epoch  11
Train prediction loss is  0.01  for epoch  12
Train prediction loss is  0.01  for epoch  13
Train prediction loss is  0.01  for epoch  14
Train prediction loss is  0.01  for epoch  15
Train prediction loss is  0.01  for epoch  16
Train prediction loss is  0.01  for epoch  17
Train prediction loss is  0.0  for epoch  18
Train prediction loss is  0.0  for epoch  19
Model Saved
Train prediction loss is  0.0  for epoch  20
Train predi

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/Implicit/Ciao/Models/NCF_"+str(10)+"_"+str(1e-3),map_location="cpu"))

<All keys matched successfully>

In [ ]:
import os
import math
import warnings
warnings.filterwarnings("ignore")
def _sample_negative(item_map,ratings):
        random.seed(10)
        """return all negative items & 100 sampled negative items"""
        interact_status = ratings.groupby('userId')['productId'].apply(set).reset_index().rename(
            columns={'productId': 'interacted_items'})
        interact_status['negative_samples'] = interact_status['interacted_items'].apply(lambda x: random.sample(set(list(range(len(item_map)))) - x, 40))
        return interact_status[['userId', 'negative_samples']]
        #interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: set(list(range(len(item_map)))) - x)
        #interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 10))
        #return interact_status[['userId', 'negative_items', 'negative_samples']]

def cal_hit_ratio(subjects,topk=10):
        """Hit Ratio @ top_K"""
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]  # golden items hit in the top_K items
        return test_in_top_k['user'].nunique() / full['user'].nunique()

def cal_ndcg(subjects,topk=10):
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]
        test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1
        return test_in_top_k['ndcg'].sum() * 1.0 / full['user'].nunique()



#model.load_state_dict(torch.load("/content/drive/MyDrive/Copy of Epinions/Models_Final/NCF_80_0.001"))
path = "/content/drive/MyDrive/Implicit/Ciao/testImplicit.csv"
#path = "/content/drive/MyDrive/Copy of Epinions/Ciao/"
loss_fn = torch.nn.BCELoss()
loss2_fn = torch.nn.L1Loss()
model.eval()
test = pd.read_csv(path)
test = test[["userId","productId","rating"]]
test_negatives = _sample_negative(list(test["productId"].unique()),test)
a = pd.merge(test, test_negatives[['userId', 'negative_samples']], on='userId')
del test_negatives
test_users,test_items,ratings,negative_users, negative_items,negative_ratings =[],[],[],[],[],[]
for row in a.itertuples():
  test_users.append(int(row.userId))
  test_items.append(int(row.productId))
  ratings.append(float(row.rating))
  for i in range(len(row.negative_samples)):
      negative_users.append(int(row.userId))
      negative_items.append(int(row.negative_samples[i]))
      negative_ratings.append(float(0))  # negative samples get 0 rating
tu,ti,tr,nu,ni,nr = torch.LongTensor(test_users), torch.LongTensor(test_items), torch.DoubleTensor(ratings), torch.LongTensor(negative_users),torch.LongTensor(negative_items),torch.DoubleTensor(negative_ratings)
predr = []
neg_predr = []
for i in range(len(tu)):
  predr.append(model.forward(tu[i],ti[i]).cpu().detach().numpy())
for i in range(len(nu)):
  neg_predr.append(model.forward(nu[i],ni[i]).cpu().detach().numpy())
subjects = [tu,ti,predr,nu,ni,neg_predr]
test_users, test_items, test_scores = subjects[0], subjects[1], subjects[2] 
neg_users, neg_items, neg_scores = subjects[3], subjects[4], subjects[5]
# the golden set
test = pd.DataFrame({'user': test_users,
                    'test_item': test_items,
                    'test_score': test_scores})
# the full set
full = pd.DataFrame({'user':  np.append(neg_users,test_users),
                  'item':  np.append(neg_items,test_items),
                  'score':  np.append(neg_scores,test_scores)})
full = pd.merge(full, test, on=['user'], how='left')
# rank the items according to the scores for each user
full['rank'] = full.groupby('user')['score'].rank(method='first', ascending=False)
full.sort_values(['user', 'rank'], inplace=True)
subjects = full
print("Hit Ratio is: ",round(cal_hit_ratio(subjects,5),2))
print("NDCG Value is: ",round(cal_ndcg(subjects,5),2))

Hit Ratio is:  0.09
NDCG Value is:  0.06


In [ ]:
top_k = full[full['rank']<=5]
top_k

user   item     score  test_item  test_score  rank
10595245     0    593  0.997066        259   0.9964019   1.0
10595246     0    593  0.997066        622   0.9966101   2.0
10595247     0    593  0.997066        493  0.99549073   3.0
10595248     0    593  0.997066        398   0.9961831   4.0
10595249     0    593  0.997066        422   0.9956721   5.0
...        ...    ...       ...        ...         ...   ...
54292073  7372  22441  0.997176      22748   0.9953135   1.0
54292074  7372  22441  0.997176      22441   0.9971764   2.0
54292075  7372  22441  0.997176      63566   0.9965649   3.0
48974045  7372   7148  0.997070      22748   0.9953135   4.0
48974046  7372   7148  0.997070      22441   0.9971764   5.0

[33565 rows x 6 columns]

In [ ]:
train_popularity = np.loadtxt("/content/drive/MyDrive/Implicit/Ciao/train_popularity.txt")
x = top_k.groupby("user")
users = list(x.groups.keys())
pops = []
for i in users:
  a = x.get_group(i)
  items = a['item'].values
  for j in items:
    s= train_popularity[j]
    pops.append(s)
print(sum(pops)/len(pops))

5.0580366453150605


In [ ]:
import os
import math
import warnings
warnings.filterwarnings("ignore")
def _sample_negative(item_map,ratings):
        random.seed(10)
        """return all negative items & 100 sampled negative items"""
        interact_status = ratings.groupby('userId')['productId'].apply(set).reset_index().rename(
            columns={'productId': 'interacted_items'})
        interact_status['negative_samples'] = interact_status['interacted_items'].apply(lambda x: random.sample(set(list(range(len(item_map)))) - x, 40))
        return interact_status[['userId', 'negative_samples']]
        #interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: set(list(range(len(item_map)))) - x)
        #interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 10))
        #return interact_status[['userId', 'negative_items', 'negative_samples']]

def cal_hit_ratio(subjects,topk=10):
        """Hit Ratio @ top_K"""
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]  # golden items hit in the top_K items
        return test_in_top_k['user'].nunique() / full['user'].nunique()

def cal_ndcg(subjects,topk=10):
        full, top_k = subjects,topk
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]
        test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1
        return test_in_top_k['ndcg'].sum() * 1.0 / full['user'].nunique()



#model.load_state_dict(torch.load("/content/drive/MyDrive/Copy of Epinions/Models_Final/NCF_80_0.001"))
path = "/content/drive/MyDrive/Implicit/Ciao/"
#path = "/content/drive/MyDrive/Copy of Epinions/Ciao/"
files = os.listdir(path)
loss_fn = torch.nn.BCELoss()
loss2_fn = torch.nn.L1Loss()
model.eval()
for f in files:
  if("test" in f):
    if("test" in f and f in ["ratings_data_test_implicit1_1_0.csv","ratings_data_test_implicit1_2_0.csv","ratings_data_test_implicit1_5_0.csv","ratings_data_test_implicit1_10_0.csv"]):
      if(f=="test.csv"):
        continue
      print(f)  
      test = pd.read_csv(path+f)
      test = test[["userId","productId","rating"]]
      test_negatives = _sample_negative(list(test["productId"].unique()),test)
      a = pd.merge(test, test_negatives[['userId', 'negative_samples']], on='userId')
      del test_negatives
      test_users,test_items,ratings,negative_users, negative_items,negative_ratings =[],[],[],[],[],[]
      for row in a.itertuples():
        test_users.append(int(row.userId))
        test_items.append(int(row.productId))
        ratings.append(float(row.rating))
        for i in range(len(row.negative_samples)):
            negative_users.append(int(row.userId))
            negative_items.append(int(row.negative_samples[i]))
            negative_ratings.append(float(0))  # negative samples get 0 rating
      tu,ti,tr,nu,ni,nr = torch.LongTensor(test_users), torch.LongTensor(test_items), torch.DoubleTensor(ratings), torch.LongTensor(negative_users),torch.LongTensor(negative_items),torch.DoubleTensor(negative_ratings)
      predr = []
      neg_predr = []
      for i in range(len(tu)):
        predr.append(model.forward(tu[i],ti[i]).cpu().detach().numpy())
      for i in range(len(nu)):
        neg_predr.append(model.forward(nu[i],ni[i]).cpu().detach().numpy())
      subjects = [tu,ti,predr,nu,ni,neg_predr]
      test_users, test_items, test_scores = subjects[0], subjects[1], subjects[2] 
      neg_users, neg_items, neg_scores = subjects[3], subjects[4], subjects[5]
      # the golden set
      test = pd.DataFrame({'user': test_users,
                          'test_item': test_items,
                          'test_score': test_scores})
      # the full set
      full = pd.DataFrame({'user':  np.append(neg_users,test_users),
                        'item':  np.append(neg_items,test_items),
                        'score':  np.append(neg_scores,test_scores)})
      full = pd.merge(full, test, on=['user'], how='left')
      # rank the items according to the scores for each user
      full['rank'] = full.groupby('user')['score'].rank(method='first', ascending=False)
      full.sort_values(['user', 'rank'], inplace=True)
      subjects = full
      print("Hit Ratio is: ",round(cal_hit_ratio(subjects,5),2))
      print("NDCG Value is: ",round(cal_ndcg(subjects,5),2))
      del full

ratings_data_test_implicit1_2_0.csv
Hit Ratio is:  0.11
NDCG Value is:  0.07
ratings_data_test_implicit1_5_0.csv
Hit Ratio is:  0.11
NDCG Value is:  0.07
ratings_data_test_implicit1_10_0.csv
Hit Ratio is:  0.12
NDCG Value is:  0.07
ratings_data_test_implicit1_1_0.csv
Hit Ratio is:  0.1
NDCG Value is:  0.06
